In [1]:
pip install librosa

  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
  Obtaining dependency information for audioread>=2.1.9 from https://files.pythonhosted.org/packages/57/8d/30aa32745af16af0a9a650115fbe81bde7c610ed5c21b381fca0196f3a7f/audioread-3.0.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.0 MB/s eta 0:00:005.4 MB/s eta 0:00:01
  Obtaining dependency information for pooch>=1.0 from https://files.pythonhosted.org/packages/1a/a5/5174dac3957ac412e80a00f30b6507031fcab7000afc9ea0ac413bddcff2/pooch-1.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for soxr>=0.3.2 from https://files.pythonhosted.org/packages/d0/79/c93daceac24cd6830333d9f3d04716115240ef3be4e8d8ece511afbcf417/soxr-0.3.7-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 21.1 MB/s eta 0:

In [8]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import argparse
import os
from glob import glob
import numpy as np
import pandas as pd
from librosa.core import resample, to_mono
from tqdm import tqdm
import wavio


def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/20),
                       min_periods=1,
                       center=True).max()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask, y_mean


def downsample_mono(path, sr):
    obj = wavio.read(path)
    wav = obj.data.astype(np.float32, order='F')
    rate = obj.rate
    try:
        channel = wav.shape[1]
        if channel == 2:
            wav = to_mono(wav.T)
        elif channel == 1:
            wav = to_mono(wav.reshape(-1))
    except IndexError:
        wav = to_mono(wav.reshape(-1))
        pass
    except Exception as exc:
        raise exc
    wav = resample(wav, orig_sr=rate, target_sr=sr)
    wav = wav.astype(np.int16)
    return sr, wav


def save_sample(sample, rate, target_dir, fn, ix):
    fn = fn.split('.wav')[0]
    dst_path = os.path.join(target_dir.split('.')[0], fn+'_{}.wav'.format(str(ix)))
    if os.path.exists(dst_path):
        return
    wavfile.write(dst_path, rate, sample)


def check_dir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)


def split_wavs(args):
    src_root = args.src_root
    dst_root = args.dst_root
    dt = args.delta_time

    # Ensure the destination directory exists
    check_dir(dst_root)

    # Iterate through each class in the source directory
    for _cls in os.listdir(src_root):
        src_dir = os.path.join(src_root, _cls)

        # Check if it's a directory
        if os.path.isdir(src_dir):
            target_dir = os.path.join(dst_root, _cls)
            check_dir(target_dir)

            # Process each .wav file in the directory
            for fn in tqdm(os.listdir(src_dir)):
                if fn.lower().endswith('.wav'):  # Check for .wav files
                    src_fn = os.path.join(src_dir, fn)
                    rate, wav = downsample_mono(src_fn, args.sr)
                    mask, y_mean = envelope(wav, rate, threshold=args.threshold)
                    wav = wav[mask]
                    delta_sample = int(dt*rate)

                    # Process and save the audio samples
                    if wav.shape[0] < delta_sample:
                        sample = np.zeros(shape=(delta_sample,), dtype=np.int16)
                        sample[:wav.shape[0]] = wav
                        save_sample(sample, rate, target_dir, fn, 0)
                    else:
                        trunc = wav.shape[0] % delta_sample
                        for cnt, i in enumerate(np.arange(0, wav.shape[0]-trunc, delta_sample)):
                            start = int(i)
                            stop = int(i + delta_sample)
                            sample = wav[start:stop]
                            save_sample(sample, rate, target_dir, fn, cnt)
        else:
            print(f"Skipping non-directory: {src_dir}")



def test_threshold(args):
    src_root = args.src_root
    wav_paths = glob('{}/**'.format(src_root), recursive=True)
    wav_path = [x for x in wav_paths if args.fn in x]
    if len(wav_path) != 1:
        print('audio file not found for sub-string: {}'.format(args.fn))
        return
    rate, wav = downsample_mono(wav_path[0], args.sr)
    mask, env = envelope(wav, rate, threshold=args.threshold)
    plt.style.use('ggplot')
    plt.title('Signal Envelope, Threshold = {}'.format(str(args.threshold)))
    plt.plot(wav[np.logical_not(mask)], color='r', label='remove')
    plt.plot(wav[mask], color='c', label='keep')
    plt.plot(env, color='m', label='envelope')
    plt.grid(False)
    plt.legend(loc='best')
    plt.show()


if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Cleaning audio data')
    
    # replace with your absolute file path here
    parser.add_argument('--src_root', type=str, default='/Users/quanhongjin/Documents/Cornell Tech/CS5785/Porject/instrument_classification_by_family',
                        help='directory of audio files in total duration')
    
    # replace with your absolute file path here
    parser.add_argument('--dst_root', type=str, default='/Users/quanhongjin/Documents/Cornell Tech/CS5785/Porject/clean',
                        help='directory to put audio files split by delta_time')
    parser.add_argument('--delta_time', '-dt', type=float, default=1.0,
                        help='time in seconds to sample audio')
    parser.add_argument('--sr', type=int, default=16000,
                        help='rate to downsample audio')

    parser.add_argument('--fn', type=str, default='3a3d0279',
                        help='file to plot over time to check magnitude')
    parser.add_argument('--threshold', type=str, default=20,
                        help='threshold magnitude for np.int16 dtype')
    args, _ = parser.parse_known_args()

    #test_threshold(args)
    split_wavs(args)

100%|█████████████████████████████████████████| 576/576 [00:08<00:00, 66.37it/s]


In [4]:
pip install wavio

  Obtaining dependency information for wavio from https://files.pythonhosted.org/packages/bf/02/40d03e99a3d2d8d1e9392f44376f470120427ffb12483579dc7e0365f712/wavio-0.0.8-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.
